
##Section 1: Imports and Parameters##

In [1]:
import numpy as np
import pennylane as qml
import random

# Lanmodulin target sequence
target_sequence = "MDRPRVIVGAAGDQVSDETLQKRYDGVSLVTVEGKEDGRIVQGLQKRDQGNLLQATLDLGKEGLRVTVEFGKEDEMLIGLKHRDQGNLLQVSLELGKKH"

# Parameters for sequence generation
sequence_length = len(target_sequence)  # Length of generated sequences
population_size = 50  # Number of sequences in the population
num_generations = 100  # Number of generations to evolve
mutation_rate = 0.3  # Probability of mutating a residue
alphabet = "ACDEFGHIKLMNPQRSTVWY"  # All possible amino acids


Grouping Function for Sequence Reduction

In [2]:
def group_sequence(sequence, group_size):
    """
    Group residues into blocks to reduce qubit requirements.
    """
    return [sequence[i:i + group_size] for i in range(0, len(sequence), group_size)]


### Quantum Fitness Function

In [4]:
def quantum_fitness_function(sequence, target):
    """
    Compute quantum similarity between a sequence and the target.
    """
    group_size = 5  # Group size to reduce qubits
    grouped_sequence = group_sequence(sequence, group_size)
    grouped_target = group_sequence(target, group_size)

    num_qubits = len(grouped_sequence)
    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def overlap_circuit(params1, params2):
        for i in range(len(params1)):
            qml.RX(params1[i], wires=i)
        qml.adjoint(lambda: [qml.RX(params2[j], wires=j) for j in range(len(params2))])()
        return qml.probs(wires=range(num_qubits))

    # Encode the sequences as rotation parameters
    params1 = [np.pi / 2 if "D" in block or "E" in block else 0 for block in grouped_sequence]
    params2 = [np.pi / 2 if "D" in block or "E" in block else 0 for block in grouped_target]

    probabilities = overlap_circuit(params1, params2)

    # Compute similarity, normalized to a positive range
    similarity = 1.0 - np.linalg.norm(probabilities - np.ones(len(probabilities)) / len(probabilities))
    return max(similarity, 0)  # Ensure non-negative fitness


### Genetic Algorithm 

In [5]:
def genetic_algorithm(target, population_size, num_generations, mutation_rate):
    """
    Run a genetic algorithm to evolve sequences toward the target.
    """
    population = ["".join(random.choices(alphabet, k=len(target))) for _ in range(population_size)]
    best_sequence = None
    best_fitness = -1
    no_improvement_generations = 0
    patience = 10  # Stop after 10 generations without improvement

    for generation in range(num_generations):
        fitness_scores = [quantum_fitness_function(seq, target) for seq in population]

        # Debugging: Log all fitness scores
        print(f"Generation {generation + 1}: Fitness Scores = {fitness_scores}")
        print(f"Sum of Fitness Scores: {sum(fitness_scores)}")

        #Handle invalid fitness scores
        if sum(fitness_scores) <= 0:
            fitness_scores = [1.0] * len(fitness_scores)  # Assign equal fitness to all sequences
            print("Fallback: Assigning uniform fitness scores to all sequences.")

        # Check for invalid fitness scores
        if all(score <= 0 for score in fitness_scores):
            raise ValueError("All fitness scores are zero or negative. Cannot proceed with selection.")

        max_fitness = max(fitness_scores)
        if max_fitness > best_fitness:
            best_fitness = max_fitness
            best_sequence = population[fitness_scores.index(max_fitness)]
            no_improvement_generations = 0
        else:
            no_improvement_generations += 1

        if no_improvement_generations >= patience:
            print("Early stopping: No improvement for 10 generations.")
            break

        # Parent selection with fallback to uniform selection
        if sum(fitness_scores) == 0:
            selected_parents = random.choices(population, k=population_size)
        else:
            selected_parents = random.choices(population, weights=fitness_scores, k=population_size)

        # Generate new population with crossover and mutation
        new_population = []
        for i in range(0, population_size, 2):
            parent1, parent2 = selected_parents[i], selected_parents[(i + 1) % population_size]
            split = random.randint(1, len(parent1) - 1)
            child = parent1[:split] + parent2[split:]

            # Mutation
            child = list(child)
            for j in range(len(child)):
                if random.random() < mutation_rate:
                    child[j] = random.choice(alphabet)
            new_population.append("".join(child))

        population = new_population
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")
        print(f"Best Sequence: {best_sequence}")

    return best_sequence, best_fitness


In [6]:
# Run the genetic algorithm
best_sequence, best_fitness = genetic_algorithm(
    target_sequence, population_size, num_generations, mutation_rate
)

print("\nFinal Best Sequence:")
print(best_sequence)
print("Fitness:", best_fitness)


Generation 1: Fitness Scores = [0.9116170473129371, 0.9844055474387976, 0.9558166170864703, 0.9558166170864703, 0.9779245028213724, 0.9779245028213724, 0.9889946995414213, 0.9687652625161729, 0.9687652625161729, 0.9375076298602494, 0.9844055474387976, 0.9687652625161729, 0.9779245028213724, 0.9844055474387976, 0.9687652625161729, 0.9375076298602494, 0.9889946995414213, 0.9687652625161729, 0.9844055474387976, 0.9116170473129371, 0.9844055474387976, 0.9687652625161729, 0.9558166170864703, 0.9889946995414213, 0.9889946995414213, 0.9687652625161729, 0.9889946995414213, 0.9889946995414213, 0.9687652625161729, 0.9687652625161729, 0.9558166170864703, 0.9375076298602494, 0.9779245028213724, 0.9844055474387976, 0.9844055474387976, 0.9687652625161729, 0.9116170473129371, 0.9558166170864703, 0.9558166170864703, 0.9945627301144238, 0.9922487754558654, 0.9558166170864703, 0.9889946995414213, 0.9889946995414213, 0.9375076298602494, 0.9844055474387976, 0.9558166170864703, 0.9779245028213724, 0.875003